In [1]:
import torch 
import nltk
from nltk.corpus import stopwords
import os
import pandas as pd
import numpy as np

In [2]:
import string
import re

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
csv_dataset=pd.read_csv("spam.csv")

In [5]:
csv_dataset.shape

(5572, 5)

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tuhin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
stop_words=stopwords.words("english")

In [9]:
dataset=[]

In [10]:
for i,row in csv_dataset.iterrows():
    dataset.append(row["v2"])

In [11]:
len(dataset)

5572

In [12]:
clean_text1=[]
def clean1(dataset):
    for w in dataset:
        if w not in stopwords.words("english"):
            clean_text1.append(w.split(" "))
clean1(dataset)

In [13]:
len(clean_text1)

5572

In [14]:
clean_text2=[]
def lower_clean(dataset):
    for w in dataset:
        new=[]
        for word in w:
            new.append(word.lower())
        clean_text2.append(new)
lower_clean(clean_text1)

In [15]:
len(clean_text2)

5572

In [16]:
clean_text2[0]

['go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet...',
 'cine',
 'there',
 'got',
 'amore',
 'wat...']

In [17]:
res=[]
def reg_exp(dataset):
    for w in dataset:
        new=[]
        for word in w:
                re_word=re.sub("[@]","",word)
                re_word=re.sub("[!]","",re_word)
                re_word=re.sub("[_]","",re_word)
                re_word=re.sub("[/]","",re_word)
                re_word=re.sub("[()]","",re_word)
                re_word=re.sub("[-]","",re_word)
                re_word=re.sub("[:]","",re_word)
                re_word=re.sub("[subject]","",re_word)
                re_word=re.sub("[,]","",re_word)
                re_word=re.sub("[.]","",re_word)
                re_word=re.sub("[;]","",re_word)
                re_word=re.sub("['']","",re_word)
                re_word=re.sub(r'[0-9]+',"", re_word)
                new.append(re_word)
        res.append(new)
reg_exp(clean_text2)


In [18]:
csv_dataset["v2"][786]

'It does it on its own. Most of the time it fixes my spelling. But sometimes it gets a completely diff word. Go figure'

In [19]:
res[0]

['go',
 'nil',
 'rong',
 'poin',
 'razy',
 'availal',
 'only',
 'in',
 'gi',
 'n',
 'gra',
 'world',
 'la',
 '',
 'ff',
 'in',
 'hr',
 'go',
 'amor',
 'wa']

In [20]:
mydata=[]
for j in range(len(clean_text2)):
    mystr=" "
    for i in clean_text2[j]:
        mystr+=" "+i
    mydata.append(mystr)

In [21]:
len(mydata)

5572

In [22]:
csv_dataset["v2"][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [23]:
mydata[0]

'  go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...'

In [24]:
for k in range(len(mydata)):
    csv_dataset["v2"][k]=mydata[k]

In [25]:
len(res)

5572

In [26]:
csv_dataset.to_csv("cleaned.csv")

In [27]:
df=pd.read_csv("cleaned.csv")

In [28]:
df["v1"].head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: v1, dtype: object

In [29]:
Vect=CountVectorizer(max_features=3000)
X=Vect.fit_transform(df["v2"]).toarray()

In [30]:
type(X)

numpy.ndarray

In [31]:
label=[]

In [32]:
X.shape

(5572, 3000)

In [33]:
for i,row in csv_dataset.iterrows():
    if row["v1"]=="ham":
        label.append(0)
    else:
        label.append(1)

In [34]:
len(label)

5572

In [35]:
y=np.array(label)

In [36]:
x_train,x_test,y_train,y_test=train_test_split(X,label,test_size=0.2)

In [37]:
len(X),len(y)

(5572, 5572)

In [38]:
clf=MultinomialNB()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
ac=accuracy_score(y_pred,y_test)

In [39]:
print(ac)

0.9829596412556054


In [49]:
test=["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
test_dt=Vect.transform(test)
pred=clf.predict(test_dt)[0]

In [50]:
print(pred)

1


In [44]:
import pickle
pickle.dump(Vect,open("CountVect.pkl","wb"))
pickle.dump(clf,open("model.pkl","wb"))
